## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barrow,US,71.2906,-156.7887,33.82,80,75,9.22,broken clouds
1,1,Jamestown,US,42.0970,-79.2353,73.04,54,0,3.44,clear sky
2,2,Luwuk,ID,-0.9516,122.7875,84.85,75,32,2.28,scattered clouds
3,3,Vardo,NO,70.3705,31.1107,37.38,81,100,9.22,overcast clouds
4,4,Taksimo,RU,56.3444,114.8794,56.93,46,82,11.65,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Luwuk,ID,-0.9516,122.7875,84.85,75,32,2.28,scattered clouds
12,12,Manado,ID,1.4870,124.8455,83.17,83,40,2.30,scattered clouds
17,17,Talcher,IN,20.9500,85.2167,83.97,76,60,4.14,broken clouds
25,25,Damaturu,NG,11.7470,11.9608,84.96,23,29,5.19,scattered clouds
31,31,Faanui,PF,-16.4833,-151.7500,80.46,72,22,7.85,few clouds
32,32,Kloulklubed,PW,7.0419,134.2556,81.10,72,8,8.10,clear sky
51,51,Umm Lajj,SA,25.0213,37.2685,83.66,51,0,10.11,clear sky
52,52,Kapaa,US,22.0752,-159.3190,82.38,73,0,9.22,clear sky
54,54,Hithadhoo,MV,-0.6000,73.0833,82.80,76,100,11.88,overcast clouds
61,61,Hilo,US,19.7297,-155.0900,86.29,72,100,14.97,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                113
City                   113
Country                113
Lat                    113
Lng                    113
Max Temp               113
Humidity               113
Cloudiness             113
Wind Speed             113
Current Description    113
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Luwuk,ID,84.85,scattered clouds,-0.9516,122.7875,
12,Manado,ID,83.17,scattered clouds,1.4870,124.8455,
17,Talcher,IN,83.97,broken clouds,20.9500,85.2167,
25,Damaturu,NG,84.96,scattered clouds,11.7470,11.9608,
31,Faanui,PF,80.46,few clouds,-16.4833,-151.7500,
32,Kloulklubed,PW,81.10,clear sky,7.0419,134.2556,
51,Umm Lajj,SA,83.66,clear sky,25.0213,37.2685,
52,Kapaa,US,82.38,clear sky,22.0752,-159.3190,
54,Hithadhoo,MV,82.80,overcast clouds,-0.6000,73.0833,
61,Hilo,US,86.29,overcast clouds,19.7297,-155.0900,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
        lat = row["Lat"]
        lng = row["Lng"]
        
        params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
        hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        except (IndexError):
            print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Luwuk,ID,84.85,scattered clouds,-0.9516,122.7875,Hotel Santika Luwuk
12,Manado,ID,83.17,scattered clouds,1.4870,124.8455,Aryaduta Hotel
17,Talcher,IN,83.97,broken clouds,20.9500,85.2167,Hotel Green Park
25,Damaturu,NG,84.96,scattered clouds,11.7470,11.9608,"Hayas Guest Inn, Central Roundabout, Gujba Road"
31,Faanui,PF,80.46,few clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))